In [ ]:
#VOTING CLASSIFIER

In [2]:
import numpy as np
from gensim.models import Word2Vec
import pickle

# Load the saved Word2Vec model
word2vec_model_path = "C:/Users/ncssa/Downloads/word2vec_model.model"
word2vec_model = Word2Vec.load(word2vec_model_path)

# Load the saved Voting Classifier model
voting_model_path = "C:/Users/ncssa/Downloads/voting_classifier_softtextmodel.pkl"
with open(voting_model_path, 'rb') as file:
    voting_model = pickle.load(file)

# Function to create a vector for a sentence using Word2Vec
def sentence_to_vector(sentence, word2vec_model, vector_size):
    words = sentence.split()
    word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(vector_size)

# Input text
input_text = ["It's a great product."]

# Generate Word2Vec vectors for the input text
vector_size = word2vec_model.vector_size  # Dimension of Word2Vec vectors
input_vectors = np.array([sentence_to_vector(sentence, word2vec_model, vector_size) for sentence in input_text])

# Step 1: Get predictions from individual models in the VotingClassifier
rf_pred = voting_model.estimators_[0].predict(input_vectors)[0]
lr_pred = voting_model.estimators_[1].predict(input_vectors)[0]
svm_pred = voting_model.estimators_[2].predict(input_vectors)[0]

# Step 2: Get the final prediction from the Voting Classifier
final_pred = voting_model.predict(input_vectors)[0]  # Get the class prediction (numeric)
predicted_probabilities = voting_model.predict_proba(input_vectors)[0]  # Get class probabilities

# Ensure the predictions are numeric for consistent mapping
if isinstance(final_pred, str):
    final_pred = int(final_pred == "OR")

# Map the predicted class to labels (numeric)
label_mapping = {0: "CG", 1: "OR"}  # Map 0 to "CG" and 1 to "OR"
final_pred_label = label_mapping[final_pred]

# Print predictions from individual models and the final prediction
print(f"Random Forest Prediction: {label_mapping[rf_pred]}")
print(f"Logistic Regression Prediction: {label_mapping[lr_pred]}")
print(f"SVM Prediction: {label_mapping[svm_pred]}")
print(f"Voting Classifier Prediction: {final_pred_label}")
print(f"Class Probabilities: {predicted_probabilities}")


Random Forest Prediction: OR
Logistic Regression Prediction: CG
SVM Prediction: CG
Voting Classifier Prediction: CG
Class Probabilities: [0.6533393 0.3466607]


In [3]:
#DEEP LEARNING LSTM

In [4]:
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder

# Download necessary NLTK resources
try:
    nltk.download('punkt_tab')  # For newer NLTK versions
except:
    nltk.download('punkt')  # Fallback for classic tokenizer

# Load the saved Word2Vec model
word2vec_model = Word2Vec.load("C:/Users/ncssa/Downloads/word2vec_model.model")

# Load the saved LSTM model
lstm_model = load_model('lstm_model.h5')

# Initialize the LabelEncoder
label_encoder = LabelEncoder()
try:
    # Attempt to load classes from the saved file
    label_encoder.classes_ = np.load('label_classes.npy', allow_pickle=True)
except FileNotFoundError:
    # Fallback to manually define classes
    print("Warning: 'label_classes.npy' not found. Define classes manually.")
    class_labels = ['OG', 'CG']  # Define classes as Original (OG) and Computer Generated (CG)
    label_encoder.fit(class_labels)

# Review to classify
review = "It's a great product."

# Tokenize the review
tokens = word_tokenize(review.lower())  # Convert to lowercase and tokenize
print("Tokens:", tokens)

# Convert tokens to Word2Vec vectors
vectors = [word2vec_model.wv[word] for word in tokens if word in word2vec_model.wv]

# Aggregate word vectors into a sentence vector
if vectors:
    sentence_vector = np.mean(vectors, axis=0)
else:
    # Handle case where no tokens are in the Word2Vec vocabulary
    sentence_vector = np.zeros(word2vec_model.vector_size)

# Reshape the sentence vector for LSTM input
sentence_vector_reshaped = sentence_vector.reshape(1, 1, -1)  # (1 sample, 1 timestep, vector_size)

# Predict the class using the LSTM model
prediction = lstm_model.predict(sentence_vector_reshaped)

# Get the predicted class index
predicted_class = np.argmax(prediction, axis=1)

# Decode the predicted class to 'OG' or 'CG'
decoded_class = label_encoder.inverse_transform(predicted_class)

# Output the prediction
print("Predicted Class:", decoded_class[0])


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ncssa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Tokens: ['it', "'s", 'a', 'great', 'product', '.']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 461ms/step
Predicted Class: CG


In [5]:
#Combining BERT and LSTM for hybrid model predictions.

In [7]:
from tensorflow.keras.models import load_model
from transformers import AutoTokenizer, TFAutoModel
import numpy as np

# Load the `.keras` file
loaded_model = load_model("C:/Users/ncssa/Downloads/bert_lstm_model.keras")
print("Model loaded successfully.")

# Load tokenizer and BERT model
bert_model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = TFAutoModel.from_pretrained(bert_model_name)

# Input text
input_text = ["It's a great product."]

# Tokenize the input text
max_length = 128
input_tokens = tokenizer(input_text, padding=True, truncation=True, return_tensors="tf", max_length=max_length)

# Generate BERT embeddings
input_embeddings = bert_model(input_tokens)
input_pooled_output = input_embeddings['pooler_output']

# Reshape for LSTM input
input_embeddings_reshaped = input_pooled_output[:, np.newaxis, :]

# Predict using the trained model
predictions = loaded_model.predict(input_embeddings_reshaped)

# Decode predictions
predicted_class = np.argmax(predictions, axis=1)[0]
label_mapping = {0: "CG", 1: "OR"}  # Adjust based on your dataset
predicted_label = label_mapping[predicted_class]
print(f"Predicted sentiment: {predicted_label}")


Model loaded successfully.


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 696ms/step
Predicted sentiment: CG


In [8]:
#Pre-trained models for text classification.

In [9]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
#Import Required Libraries
from transformers import BertTokenizer, BertForSequenceClassification
import torch


# Path to the saved model and tokenizer
save_path = "C:/Users/ncssa/Downloads/bert_trained_model"

# Load the model and tokenizer
loaded_model = AutoModelForSequenceClassification.from_pretrained(save_path)
loaded_tokenizer = AutoTokenizer.from_pretrained(save_path)

print("Model and tokenizer loaded successfully.")

# Sample text for prediction
text = "It's a great product"

# Tokenize the input text
inputs = loaded_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Get model prediction
with torch.no_grad():
    outputs = loaded_model(**inputs)

# Extract the predicted class (the class with the highest score)
predicted_class = torch.argmax(outputs.logits, dim=1).item()

# Map numeric output to 'OG' or 'CG'
class_map = {0: 'OG', 1: 'CG'}

# Print the predicted label
predicted_label = class_map.get(predicted_class, "Unknown")
print(f"Predicted label: {predicted_label}")


Model and tokenizer loaded successfully.
Predicted label: CG


In [10]:
#Pre-trained models for sentiment classification.

In [12]:

from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Path to the saved model and tokenizer
save_path = "C:/Users/ncssa/Downloads/sentiment_analysis_finalmodel"

# Load the model and tokenizer
loaded_model = AutoModelForSequenceClassification.from_pretrained(save_path)
loaded_tokenizer = AutoTokenizer.from_pretrained(save_path)

print("Model and tokenizer loaded successfully.")

# Sample text for prediction
text = "It's a great product"

# Tokenize the input text
inputs = loaded_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Get model prediction
with torch.no_grad():
    outputs = loaded_model(**inputs)

# Extract the predicted class (the class with the highest score)
predicted_class = torch.argmax(outputs.logits, dim=1).item()

# Map numeric output to 'OG' or 'CG'
class_map = {0: 'negative', 1: 'positive'}

# Print the predicted label
predicted_label = class_map.get(predicted_class, "Unknown")
print(f"Predicted label: {predicted_label}")


Model and tokenizer loaded successfully.
Predicted label: positive
